<a href="https://colab.research.google.com/github/Leah-Wu/SI650/blob/main/Recommendor_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import json
import ast
import numpy as np 
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

In [1]:
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
!gzip -d reviews_Electronics_5.json.gz
!gzip -d meta_Electronics.json.gz

--2020-12-11 22:38:17--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz’

reviews_Electronics 100%[===================>] 472.88M  7.03MB/s    in 87s     

2020-12-11 22:39:45 (5.41 MB/s) - ‘reviews_Electronics_5.json.gz’ saved [495854086/495854086]

--2020-12-11 22:39:45--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186594679 (178M) [application/x-gzip]
Saving to: ‘meta_Electronics.json.gz’

meta_Electronics.js 100%[===

In [2]:
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Electronics.csv

--2020-12-11 22:40:30--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Electronics.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318766497 (304M) [text/csv]
Saving to: ‘ratings_Electronics.csv’

ratings_Electronics 100%[===================>] 304.00M  8.28MB/s    in 49s     

2020-12-11 22:41:19 (6.15 MB/s) - ‘ratings_Electronics.csv’ saved [318766497/318766497]



In [11]:
def merge_data(reviews_json_file, meta_json_file):
  reviews_Musical_Instruments_5 = pd.read_json(reviews_json_file,lines=True)
  meta_list = []
  with open(meta_json_file) as f:
    for line in f.readlines():
      line = json.dumps(ast.literal_eval(line))
      d = json.loads(line)
      if 'title' not in d:
        continue
      else:
        title = d['title']
        product_id = d['asin']
        img_url = d['imUrl']
    
        meta = []
        
        meta.append(product_id)
        meta.append(title)
        meta.append(img_url)

        meta_list.append(meta)
  column_names = ['asin','title','image_url']
  meta = pd.DataFrame(meta_list, columns=column_names)
  entire_merged_df = reviews_Musical_Instruments_5[['asin','overall','unixReviewTime', 'reviewText']].merge(meta[['asin', 'title','image_url']], on='asin', how='inner')
  entire_merged_df['reviewText'] = entire_merged_df['reviewText'].str.lower()
  return entire_merged_df

In [12]:
reviews_json_file = 'reviews_Electronics_5.json'
meta_json_file = 'meta_Electronics.json'
df = merge_data(reviews_json_file, meta_json_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1643686 entries, 0 to 1643685
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   asin            1643686 non-null  object
 1   overall         1643686 non-null  int64 
 2   unixReviewTime  1643686 non-null  int64 
 3   reviewText      1643686 non-null  object
 4   title           1643686 non-null  object
 5   image_url       1643686 non-null  object
dtypes: int64(2), object(4)
memory usage: 87.8+ MB


In [8]:
def popularity_recommend(merged_df, num):
    # get each product's latest released review time
    latest_time_df = merged_df[merged_df.groupby('asin')['unixReviewTime'].transform('max') == merged_df['unixReviewTime']]
    # get each product's mean rating
    filtered_df = merged_df.groupby(['asin','title','image_url'])['overall'].mean()
    filtered_df = filtered_df.reset_index(drop=False)
    # merge two df and sort by ratings then review time
    result = filtered_df[['asin','title','image_url','overall']].merge(latest_time_df[['asin','unixReviewTime']], on='asin', how='inner')
    result = result.sort_values(['overall', 'unixReviewTime'], ascending=[False, False])
    # each row: asin, title, image_url, overall, unixReviewTime, 
    return result.head(num)

In [13]:
print("Recommend top 10 popular items:")
popularity_recommend(df, 10)

Recommend top 10 popular items:


,asin,title,image_url,overall,unixReviewTime
61186,B00FXBKNQC,2 Pack Solid Cordz&reg; Premium 6ft Universal ...,http://ecx.images-amazon.com/images/I/417gtbDq...,5.0,1406073600
61897,B00HFI55N2,Fotasy APP7800 Self-Retaining Auto Open Close ...,http://ecx.images-amazon.com/images/I/513HmHn5...,5.0,1406073600
62653,B00JOS04PK,[Apple Certified] NOOT&reg; MFI 1.8M/6ft 8-Pin...,http://ecx.images-amazon.com/images/I/415Zw515...,5.0,1406073600
56331,B00B502WQ4,DC power jack socket charging port for ASUS A5...,http://ecx.images-amazon.com/images/I/31e59xh-...,5.0,1405987200
57881,B00CAJOTNI,Philips BTM2180/37 Micro Music System (Black) ...,http://ecx.images-amazon.com/images/I/41FQHqlT...,5.0,1405987200
59347,B00DJERLFG,Ubiquiti Networks UniFi PRO Enterprise WiFi Sy...,http://ecx.images-amazon.com/images/I/41Yr%2BM...,5.0,1405987200
61352,B00G9G76PQ,Nikon ML-L3 Wireless Remote Control with Stora...,http://ecx.images-amazon.com/images/I/51OMtRKE...,5.0,1405987200
61633,B00GS0O5RU,Xperitus BOS-26U Hi-Speed Quality Micro USB To...,http://ecx.images-amazon.com/images/I/51WfBady...,5.0,1405987200
62030,B00HPPI0W8,Brainwavz Large Hard Headphone Case - Suitable...,http://ecx.images-amazon.com/images/I/413tk3sR...,5.0,1405987200
62187,B00I0D81OG,Sapphire Radeon R9 270 2GB GDDR5 DVI-I/DVI-D/H...,http://ecx.images-amazon.com/images/I/41yrOc30...,5.0,1405987200


In [39]:
def get_rating_matrix(rating_file_name):
    df = pd.read_csv(rating_file_name,names=['userId', 'productId','Rating','timestamp'])
    df.drop(['timestamp'], axis=1,inplace=True)
    # ###Check the distribution of the rating
    # with sns.axes_style('white'):
    #     g = sns.factorplot("Rating", data=df, aspect=2.0,kind='count')
    #     g.set_ylabels("Total number of ratings")

    #Getting the new dataframe which contains users who has given 50 or more ratings
    new_df=df.groupby("productId").filter(lambda x : x['Rating'].count() >= 50)
    # new_df=new_df.head(10000)
    ratings_matrix = new_df.pivot_table(values='Rating', index='userId', columns='productId', fill_value=0)   
    return ratings_matrix

In [40]:
def recommend(ratings_matrix, merged_df, p_id, correlation):
    # Transpose the matrix
    X = ratings_matrix.T
    SVD = TruncatedSVD(n_components=10)
    decomposed_matrix = SVD.fit_transform(X)

    #Correlation Matrix: product to product correlation
    correlation_matrix = np.corrcoef(decomposed_matrix)
    # find the position of given product in the list of all products
    product_names = list(X.index)
    product_ID = product_names.index(p_id)
    correlation_product_ID = correlation_matrix[product_ID]
    recommend = list(X.index[correlation_product_ID > correlation])
    # Removes the item already checked by the customer
    recommend.remove(p_id) 
    results = merged_df[merged_df.asin.isin(recommend)].groupby(['asin','title','image_url'])['overall'].mean()
    # results = results.sort_values('overall', ascending=False)
    # # asin, title, image_url, overall
    return results[:5]

Suppose to use "ratings_Electronics.csv" here for recommending, but it cost too much time to build up mtrix, here we used musical instruments dataset as an example.

In [31]:
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Musical_Instruments.csv

--2020-12-12 00:00:46--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Musical_Instruments.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20376609 (19M) [text/csv]
Saving to: ‘ratings_Musical_Instruments.csv’

ratings_Musical_Ins 100%[===================>]  19.43M  4.78MB/s    in 4.3s    

2020-12-12 00:00:51 (4.56 MB/s) - ‘ratings_Musical_Instruments.csv’ saved [20376609/20376609]



In [33]:
ratings_matrix = get_rating_matrix("ratings_Musical_Instruments.csv")

In [41]:
# assume customer is currently looking at product with id: 
p_id = 'B0002CZSJY'
print("Recommend you may also like:")
recommend(ratings_matrix, df, p_id, 0.2)

Recommend you may also like:


asin        title                                                                                                                                                                                                                                                                                                                                                                     image_url                                                    
B00B95REMW  SoundBot&reg; SB330 Bluetooth 3.0 Universal Audio Receiver - stream ANY music from Bluetooth enabled media player to speaker or docking station with 30 pin connector. Plug &amp; Play for Apple Docking Stations. Compatible with the New Iphone 5, Ipad 4 Mini, Other Idevices, Mp3 Bluetooth Player, Tablet, and Even Andriod or Other Smartphones w/ A2DP and AVRCP.  http://ecx.images-amazon.com/images/I/31fVkiLFkyL._SX300_.jpg    3.857143
Name: overall, dtype: float64